### Usage: Confirm validity of reference sequences vs vcf files
### Author: Temi
### Date: Tues Jan 24 2023

In [1]:
import os
import numpy as np
import cyvcf2
import kipoiseq
import pandas as pd
import multiprocessing
import subprocess

In [2]:
working_dir = '/grand/covid-ct/imlab/users/temi/projects/TFXcan/scripts/'
os.chdir(working_dir)

In [3]:
reference_genome = '/grand/covid-ct/imlab/data/hg_sequences/hg38/Homo_sapiens_assembly38.fasta'
vcf_file = '../genotypes/prj6_genotypes/merged_phased_SNPs.vcf.gz'

Use kipoiseq to read fasta file

In [4]:
class FastaStringExtractor:
    def __init__(self, fasta_file):
        import pyfaidx

        self.fasta = pyfaidx.Fasta(fasta_file)
        self._chromosome_sizes = {k: len(v) for k, v in self.fasta.items()}

    def extract(self, interval, **kwargs) -> str:
        # Truncate interval if it extends beyond the chromosome lengths.

        import kipoiseq
        chromosome_length = self._chromosome_sizes[interval.chrom]
        trimmed_interval = kipoiseq.Interval(interval.chrom,
                                    max(interval.start, 0),
                                    min(interval.end, chromosome_length),
                                    )
        # pyfaidx wants a 1-based interval
        sequence = str(self.fasta.get_seq(trimmed_interval.chrom,
                                            trimmed_interval.start + 1,
                                            trimmed_interval.stop).seq).upper()
        # Fill truncated values with N's.
        pad_upstream = 'N' * max(-interval.start, 0)
        pad_downstream = 'N' * max(interval.end - chromosome_length, 0)
        return pad_upstream + sequence + pad_downstream

    def close(self):
        return self.fasta.close()

In [60]:
fasta_extractor = FastaStringExtractor(reference_genome)

A short snippet

In [214]:
region_chr = 'chr1'
region_start = 100001
region_end = 102009
# region_chr = 'chr1'
# region_start = 10000
# region_end = 10001

region_chr = 'chr5'
region_start = 96701000
region_end = 96701500
SEQUENCE_LENGTH=len(range(region_start, region_end))

In [215]:
reg_interval = kipoiseq.Interval(region_chr, region_start, region_end).resize(SEQUENCE_LENGTH)
reference_region = fasta_extractor.extract(interval=reg_interval, anchor=[])
reference_region

'ACTGCAACCTCCACCTTCCAGGTTCAAGCGATAATCCTGCCTGAAGCCTCCCGAGTAGCTGGGATTACAGGCAAGCACCACCGCACACAGCTAATGTTTGTATTTTTAGTAGAGACAGGGTTTCACCATGTTGCCCAGGCTGGTCTGGAACTCCTGAGCTCAGGTGATCCACTTGCCTCGGCCTCCCAAAGTGGTGGGATTACAGGCATGAGCCACCATGCCTGGCCAGAAAATTTGTTTTATAACCATGTGATTACTCATCTGTACTCATCAGGAGCACAGAAATACATTCAGAGATAGGCATCATTGAGCTGCCTAACTCTGGAAGCTGTGGGGCCCAGGGCCCATCCCTATGGGGTGTGGCCCAAATTAGTCACCCTGTGCTGTTTTTGTCACAGGAGAAATTAATGTCAGTGAAACCTAGATGAGCTTTTAGTCTATTTTTAAAAAGGAATGACAATACTAATGATCAACAACAACAACAAAAAGGGATATATAAG'

## The experiment

Use cyvcf to extract vcf

There are currently about 11 million variants in this vcf file

For this purpose of this notebook, I will use the top 1 million variants (for memory sake)

In [216]:
#stop_at = 11000000
# all_variants = []

# for i, variant in enumerate(cyvcf2.cyvcf2.VCF(vcf_file)):
#     if i > stop_at:
#         break
#     else:
#         out = []
#         out.extend([variant.CHROM, variant.start, variant.end, variant.REF, variant.ALT])
#         all_variants.append(out)

In [217]:
samples = ['LuCaP_136', 'LuCaP_141']
vcf_object = cyvcf2.cyvcf2.VCF(vcf_file, samples=samples)

[W::hts_idx_load3] The index file is older than the data file: ../genotypes/prj6_genotypes/merged_phased_SNPs.vcf.gz.tbi


In [218]:
query = f'{region_chr}:{region_start}-{region_end}'
query

'chr5:96701000-96701500'

In [219]:
all_variants = []

for i, variant in enumerate(vcf_object(query)):
    out = []
    out.extend([variant.CHROM, variant.start, variant.end, variant.REF, variant.ALT, variant.gt_bases])
    all_variants.append(out)
all_variants

[W::hts_idx_load3] The index file is older than the data file: ../genotypes/prj6_genotypes/merged_phased_SNPs.vcf.gz.tbi


[['chr5', 96701090, 96701091, 'C', ['A'], array(['C|C', 'C|C'], dtype='<U3')]]

In [220]:
cmd = f'bcftools view -r {query} -s {",".join(samples)} {vcf_file}'
[re.sub('\t', '     ', b) for b in subprocess.run(cmd, shell=True, capture_output=True).stdout.decode("utf-8").split('\n')[3383:-1]]

['#CHROM     POS     ID     REF     ALT     QUAL     FILTER     INFO     FORMAT     LuCaP_136     LuCaP_141',
 'chr5     96701091     .     C     A     .     .     AC=0;AF=0.0588235;CM=104.68;AN=4     GT     0|0     0|0']

Here is what the output looks like ( I think it went over by one but it does not matter)

In [221]:
all_variants[0:3], len(all_variants)

([['chr5',
   96701090,
   96701091,
   'C',
   ['A'],
   array(['C|C', 'C|C'], dtype='<U3')]],
 1)

Extract the same regions from the reference genome

In [222]:
reference_variants_positions = [[v[0], v[1], v[2]] for v in all_variants]
reference_variants_positions[0:5]

[['chr5', 96701090, 96701091]]

In [223]:
len(reference_variants_positions)

1

In [224]:
reference_variants = []
for vpos in reference_variants_positions:
    slength = len(range(vpos[1], vpos[2]))
    reg_interval = kipoiseq.Interval(vpos[0], vpos[1], vpos[2]).resize(slength)
    reference_variant = fasta_extractor.extract(interval=reg_interval, anchor=[])
    reference_variants.append(reference_variant)

In [225]:
reference_variants

['C']

Convert to a string

In [226]:
reference_variants = ''.join(reference_variants)
reference_variants

'C'

Convert the vcf variants to a string too

In [227]:
vcf_ref_variants = ''.join([v[3] for v in all_variants])
vcf_ref_variants

'C'

Compare the reference variants with the vcf variants

In [228]:
reference_variants == vcf_ref_variants # so, the coordinates matching is correct

True

In [229]:
len(reference_variants), len(vcf_ref_variants)

(1, 1)

They are the same

###  Mutate

In [230]:
def reverse_encode_into_sequences(input_array, delim_indices):

    import copy 
    # never change this whatsoever
    A = np.array([1,0,0,0], dtype=np.float32)
    C = np.array([0,1,0,0], dtype=np.float32)
    G = np.array([0,0,1,0], dtype=np.float32)
    T = np.array([0,0,0,1], dtype=np.float32)
    seq_dict = {'A': A, 'C': C, 'G': G, 'T': T}

    keys = list(seq_dict.keys())
    values = list(seq_dict.values())
    values = [ar.tolist() for ar in values]

    if len(input_array.shape) != 3:
        raise Exception('Input array should have 3 dimensions')

    seq_list = []
    for i, na in enumerate(range(input_array.shape[1])):
        ts = input_array[:,i,:].tolist()[0]
        # look up in the dictionary
        id = values.index(ts)
        seq_list.append(keys[id])
    if delim_indices is not None:
        seq = [seq_list[i] if i in delim_indices else seq_list[i].lower() for i in range(len(seq_list)) ] #
        return(''.join(seq))
    else:
        return(''.join(seq_list))

In [231]:
import importlib
import sys
import re

In [232]:
sys.path.append('/grand/covid-ct/imlab/users/shared/enformer_pipeline/parallel_enformer/batch_utils')
import sequencesUtils

In [233]:
importlib.reload(sequencesUtils)

<module 'sequencesUtils' from '/grand/covid-ct/imlab/users/shared/enformer_pipeline/parallel_enformer/batch_utils/sequencesUtils.py'>

In [234]:
query_input = re.sub('\:|-', '_', query)
query_input

'chr5_96701000_96701500'

In [235]:
reference_sequence_encoded = sequencesUtils.extract_reference_sequence(region=query_input, fasta_func=fasta_extractor, resize_for_enformer=False, resize_length=None, print_sequence = False, write_log=None)
reference_sequence_encoded

{'sequence': array([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.],
         ...,
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]]], dtype=float32),
 'interval_object': {'chr': 'chr5', 'start': 96701001, 'end': 96701501}}

In [236]:
reference_sequence_encoded['sequence'].shape

(1, 500, 4)

In [237]:
extracted_reference = reverse_encode_into_sequences(reference_sequence_encoded['sequence'], None)
extracted_reference

'ACTGCAACCTCCACCTTCCAGGTTCAAGCGATAATCCTGCCTGAAGCCTCCCGAGTAGCTGGGATTACAGGCAAGCACCACCGCACACAGCTAATGTTTGTATTTTTAGTAGAGACAGGGTTTCACCATGTTGCCCAGGCTGGTCTGGAACTCCTGAGCTCAGGTGATCCACTTGCCTCGGCCTCCCAAAGTGGTGGGATTACAGGCATGAGCCACCATGCCTGGCCAGAAAATTTGTTTTATAACCATGTGATTACTCATCTGTACTCATCAGGAGCACAGAAATACATTCAGAGATAGGCATCATTGAGCTGCCTAACTCTGGAAGCTGTGGGGCCCAGGGCCCATCCCTATGGGGTGTGGCCCAAATTAGTCACCCTGTGCTGTTTTTGTCACAGGAGAAATTAATGTCAGTGAAACCTAGATGAGCTTTTAGTCTATTTTTAAAAAGGAATGACAATACTAATGATCAACAACAACAACAAAAAGGGATATATAAG'

In [238]:
reference_region

'ACTGCAACCTCCACCTTCCAGGTTCAAGCGATAATCCTGCCTGAAGCCTCCCGAGTAGCTGGGATTACAGGCAAGCACCACCGCACACAGCTAATGTTTGTATTTTTAGTAGAGACAGGGTTTCACCATGTTGCCCAGGCTGGTCTGGAACTCCTGAGCTCAGGTGATCCACTTGCCTCGGCCTCCCAAAGTGGTGGGATTACAGGCATGAGCCACCATGCCTGGCCAGAAAATTTGTTTTATAACCATGTGATTACTCATCTGTACTCATCAGGAGCACAGAAATACATTCAGAGATAGGCATCATTGAGCTGCCTAACTCTGGAAGCTGTGGGGCCCAGGGCCCATCCCTATGGGGTGTGGCCCAAATTAGTCACCCTGTGCTGTTTTTGTCACAGGAGAAATTAATGTCAGTGAAACCTAGATGAGCTTTTAGTCTATTTTTAAAAAGGAATGACAATACTAATGATCAACAACAACAACAAAAAGGGATATATAAG'

In [239]:
extracted_reference == reference_region

True

In [240]:
found_variants = sequencesUtils.find_variants_in_vcf_file(cyvcf2_object=vcf_object, interval_object=reference_sequence_encoded['interval_object'], samples=samples)
mapping_dictionary = sequencesUtils.create_mapping_dictionary(variants_array=found_variants, interval_start = reference_sequence_encoded['interval_object']['start'], haplotype='both', sequence_source='personalized', samples=None)
samples_variants_encoded = sequencesUtils.replace_variants_in_reference_sequence(query_sequences_encoded = reference_sequence_encoded['sequence'], mapping_dict = mapping_dictionary, samples=None)

In [241]:
samples_variants_encoded

{'LuCaP_136': {'haplotype1': array([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          ...,
          [1., 0., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 0.]]], dtype=float32),
  'haplotype2': array([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          ...,
          [1., 0., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 0.]]], dtype=float32)},
 'LuCaP_141': {'haplotype1': array([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          ...,
          [1., 0., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 0.]]], dtype=float32),
  'haplotype2': array([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          ...,
          [1., 0., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 0.]]], dtype=float32)}}

In [242]:
reverse_encode_into_sequences(samples_variants_encoded['LuCaP_136']['haplotype2'], None) == extracted_reference

True

In [243]:
sample_variant_sequence = reverse_encode_into_sequences(samples_variants_encoded['LuCaP_136']['haplotype2'], None)
sample_variant_sequence

'ACTGCAACCTCCACCTTCCAGGTTCAAGCGATAATCCTGCCTGAAGCCTCCCGAGTAGCTGGGATTACAGGCAAGCACCACCGCACACAGCTAATGTTTGTATTTTTAGTAGAGACAGGGTTTCACCATGTTGCCCAGGCTGGTCTGGAACTCCTGAGCTCAGGTGATCCACTTGCCTCGGCCTCCCAAAGTGGTGGGATTACAGGCATGAGCCACCATGCCTGGCCAGAAAATTTGTTTTATAACCATGTGATTACTCATCTGTACTCATCAGGAGCACAGAAATACATTCAGAGATAGGCATCATTGAGCTGCCTAACTCTGGAAGCTGTGGGGCCCAGGGCCCATCCCTATGGGGTGTGGCCCAAATTAGTCACCCTGTGCTGTTTTTGTCACAGGAGAAATTAATGTCAGTGAAACCTAGATGAGCTTTTAGTCTATTTTTAAAAAGGAATGACAATACTAATGATCAACAACAACAACAAAAAGGGATATATAAG'

In [244]:
extracted_reference

'ACTGCAACCTCCACCTTCCAGGTTCAAGCGATAATCCTGCCTGAAGCCTCCCGAGTAGCTGGGATTACAGGCAAGCACCACCGCACACAGCTAATGTTTGTATTTTTAGTAGAGACAGGGTTTCACCATGTTGCCCAGGCTGGTCTGGAACTCCTGAGCTCAGGTGATCCACTTGCCTCGGCCTCCCAAAGTGGTGGGATTACAGGCATGAGCCACCATGCCTGGCCAGAAAATTTGTTTTATAACCATGTGATTACTCATCTGTACTCATCAGGAGCACAGAAATACATTCAGAGATAGGCATCATTGAGCTGCCTAACTCTGGAAGCTGTGGGGCCCAGGGCCCATCCCTATGGGGTGTGGCCCAAATTAGTCACCCTGTGCTGTTTTTGTCACAGGAGAAATTAATGTCAGTGAAACCTAGATGAGCTTTTAGTCTATTTTTAAAAAGGAATGACAATACTAATGATCAACAACAACAACAAAAAGGGATATATAAG'

In [245]:
for i in range(len(extracted_reference)):
    if extracted_reference[i] != sample_variant_sequence[i]:
        print(f"index {i}: {extracted_reference[i]} != {sample_variant_sequence[i]}")

In [246]:
mapping_dictionary

{'positions': (90,),
 'LuCaP_136': {'haplotype1': (array([0., 1., 0., 0.], dtype=float32),),
  'haplotype2': (array([0., 1., 0., 0.], dtype=float32),)},
 'LuCaP_141': {'haplotype1': (array([0., 1., 0., 0.], dtype=float32),),
  'haplotype2': (array([0., 1., 0., 0.], dtype=float32),)}}

## Here, I extend the above to the GEUVADIS dataset which is split across different chromosomes :(

In [248]:
reference_genome = '/grand/covid-ct/imlab/data/hg_sequences/hg38/Homo_sapiens_assembly38.fasta'
vcf_files_dir = '/grand/covid-ct/imlab/data/GEUVADIS/vcf_snps_only'

In [249]:
chromosomes = [f'chr{i}' for i in list(range(1, 23))]
chromosomes.extend(['chrX', 'chrY', 'chrM'])
chromosomes[0:5]

['chr1', 'chr2', 'chr3', 'chr4', 'chr5']

In [250]:
fasta_extractor = FastaStringExtractor(reference_genome)

In [247]:
variants_status = {}

for chr in chromosomes:
    print(chr)
    vcf_f = f'{vcf_files_dir}/ALL.{chr}.shapeit2_integrated_SNPs_v2a_27022019.GRCh38.phased.vcf.gz'
    if os.path.isfile(vcf_f):
        chr_variants = []
        for i, variant in enumerate(cyvcf2.cyvcf2.VCF(vcf_f)):
            if i > 50000:
                break
            else:
                out = []
                out.extend([variant.CHROM, variant.start, variant.end, variant.REF, variant.ALT])
                chr_variants.append(out)
    
    chr_reference_variants_positions = [[v[0], v[1], v[2]] for v in chr_variants]
    print(chr_reference_variants_positions[1:3])

    chr_reference_variants = []
    for vpos in chr_reference_variants_positions:
        slength = len(range(vpos[1], vpos[2]))
        reg_interval = kipoiseq.Interval(vpos[0], vpos[1], vpos[2]).resize(slength)
        reference_region = fasta_extractor.extract(interval=reg_interval, anchor=[])
        chr_reference_variants.append(reference_region)

    chr_reference_variants = ''.join(chr_reference_variants)
    chr_vcf_ref_variants = ''.join([v[3] for v in chr_variants])

    status = chr_reference_variants == chr_vcf_ref_variants
    print(f'    {status}')

    variants_status[chr] = status
    

NameError: name 'chromosomes' is not defined

## Confirm bin validity